In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 5


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-06 13:18:52] ax.service.ax_client: Generated new trial 80 with parameters {'Drug': 94, 'SL_1': 72, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 0, 'S_3': 1, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

7 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,1,0
1,1,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
2            82    90    34     0     0     0     0     0    0    0  ...    0   
3            83    90    34     0     0     0     0     0    0    0  ...    0   
4            84    89    73     0     0     0     0     0    0    0  ...    0   
5            85    89    73     0     0     0     0     0    0    0  ...    0   
6            86    89    34     0     0     0     0     0    0    0  ...    0   
7            87    90    34     0     0     0     0     0    0    0  ...    0   
8            88    90    34     0     0     0     0     0    0    0  ...    0   
9            89    89    73     0     0     0     0     0    0    0  ...    0   
12           92   100    71     0     0     0     0     0    0    0  ...    0   
14           94    53    75     0     0     0     0     0    0    0  ...    0   
15           95    89    73     0     0     0     0     0    0    0  ...    0   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,1,0
1,1,0,0,0
2,0,0,0,100
3,0,0,0,100
4,0,0,0,100
5,0,0,0,100
6,0,0,0,100
7,0,0,0,100
8,0,0,0,100
9,0,0,0,100


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0000,16.000000,16.000000
mean,87.500000,0.597562,0.402438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093750,0.0625,0.156250,0.687500
std,4.760952,0.097283,0.097283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271953,0.2500,0.352077,0.478714
min,80.000000,0.414000,0.274000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
25%,83.750000,0.549000,0.275500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
50%,87.500000,0.566000,0.434000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,1.000000
75%,91.250000,0.724500,0.451000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,1.000000
max,95.000000,0.726000,0.586000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0000,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0000,16.000000,16.000000
mean,87.500000,88.375000,61.562500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0625,0.187500,68.750000
std,4.760952,10.019148,19.575388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.341565,0.2500,0.403113,47.871355
min,80.000000,53.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
25%,83.750000,89.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
50%,87.500000,89.500000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,100.000000
75%,91.250000,91.000000,73.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,100.000000
max,95.000000,100.000000,88.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0000,1.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.00000,16.000000,16.0
mean,87.500000,209.146875,140.853125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.750000,62.5,156.25000,687.500000,0.0
std,4.760952,34.049075,34.049075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,271.952815,250.0,352.07717,478.713554,0.0
min,80.000000,144.900000,95.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0
25%,83.750000,192.150000,96.425000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0
50%,87.500000,198.100000,151.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,1000.000000,0.0
75%,91.250000,253.575000,157.850000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,1000.000000,0.0
max,95.000000,254.100000,205.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.000000,1000.0,1000.00000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.3
SL_1        2.3
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         1.5
S_2         1.0
S_3         2.5
Water      11.0
Solvent     0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_5_OT_2_protocol_1.py',
 'protocol/iteration_5_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,80,0.566,0.434,0,0,0,0,0,0,0,0,0.0,0,1.0,0,2,0
1,81,0.566,0.434,0,0,0,0,0,0,0,0,1.0,0,0.0,0,2,0
2,82,0.726,0.274,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
3,83,0.726,0.274,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
4,84,0.549,0.451,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
5,85,0.549,0.451,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
6,86,0.724,0.276,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
7,87,0.726,0.274,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
8,88,0.726,0.274,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0
9,89,0.549,0.451,0,0,0,0,0,0,0,0,0.0,0,0.0,1,1,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,237.419859,0.208284
1,Good,A2,241.176594,0.231287
2,Good,A3,231.530937,0.235615
3,Good,A4,275.167677,0.187831
4,Good,A5,267.268946,0.213713
5,Caution,A6,283.765489,0.275259
6,Good,B1,220.569205,0.138947
7,Caution,B2,226.816001,0.279084
8,Caution,B3,214.866408,0.202467
9,Caution,B4,224.529784,0.202474


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,236.709130,4.861947,0.225062,0.014691,1
1,Formulation 2,271.218312,5.585246,0.200772,0.018301,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,273.487817,18.362315,0.312513,0.206909,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,80,0.566,0.434,0,0,0,0,0,0,0,...,1.0,0,2,0,Formulation 1,236.709130,4.861947,0.225062,0.014691,1
1,81,0.566,0.434,0,0,0,0,0,0,0,...,0.0,0,2,0,Formulation 2,271.218312,5.585246,0.200772,0.018301,1
2,82,0.726,0.274,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,83,0.726,0.274,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,84,0.549,0.451,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,85,0.549,0.451,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,86,0.724,0.276,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,87,0.726,0.274,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,88,0.726,0.274,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,89,0.549,0.451,0,0,0,0,0,0,0,...,0.0,1,1,0,Formulation 10,273.487817,18.362315,0.312513,0.206909,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,80,0.566,0.434,0,0,0,0,0,0,0,...,2,0,Formulation 1,236.709130,4.861947,0.225062,0.014691,1,1132.0,0
1,81,0.566,0.434,0,0,0,0,0,0,0,...,2,0,Formulation 2,271.218312,5.585246,0.200772,0.018301,1,1132.0,0
2,82,0.726,0.274,0,0,0,0,0,0,0,...,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,83,0.726,0.274,0,0,0,0,0,0,0,...,1,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,84,0.549,0.451,0,0,0,0,0,0,0,...,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
5,85,0.549,0.451,0,0,0,0,0,0,0,...,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,86,0.724,0.276,0,0,0,0,0,0,0,...,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,87,0.726,0.274,0,0,0,0,0,0,0,...,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,88,0.726,0.274,0,0,0,0,0,0,0,...,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,89,0.549,0.451,0,0,0,0,0,0,0,...,1,0,Formulation 10,273.487817,18.362315,0.312513,0.206909,1,1098.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.236709,0.004862,0.566,0.0,0.225062,0.014691,0.166667
1,1,0.271218,0.005585,0.566,0.0,0.200772,0.018301,0.166667
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,1,0.273488,0.018362,0.549,0.0,0.312513,0.206909,0.083333


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.236709,0.004862,0.566,0.0,0.225062,0.014691,0.166667
1,1,0.271218,0.005585,0.566,0.0,0.200772,0.018301,0.166667
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,1,0.273488,0.018362,0.549,0.0,0.312513,0.206909,0.083333


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,80,0.566,0.434,0,0,0,0,0,0,0,...,0.166667,0,Formulation 1,0.236709,0.004862,0.225062,0.014691,1,0.566,0
1,81,0.566,0.434,0,0,0,0,0,0,0,...,0.166667,0,Formulation 2,0.271218,0.005585,0.200772,0.018301,1,0.566,0
2,82,0.726,0.274,0,0,0,0,0,0,0,...,0.083333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,83,0.726,0.274,0,0,0,0,0,0,0,...,0.083333,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,84,0.549,0.451,0,0,0,0,0,0,0,...,0.083333,0,Formulation 5,1.000000,0.000000,1.000000,0.000000,0,0.000,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 80 with data: {'Solu': (0.566, 0), 'Size': (0.236709, 0.004862), 'PDI': (0.225062, 0.014691)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 81 with data: {'Solu': (0.566, 0), 'Size': (0.271218, 0.005585), 'PDI': (0.200772, 0.018301)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 82 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 83 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 84 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 85 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 15:19:44] ax.service.ax_client: Completed trial 86 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 15:19:44] ax.